Выполнялось в Pycharm


In [ ]:
import pandas as pd
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from scipy.stats import levene
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from math import factorial as f

from colorama import init, Fore, Back, Style

init()

df = pd.read_csv('/content/apple_quality.csv')

print(df)
print(df.describe())

print(df.isna().sum())
df.dropna(inplace=True)

print(f"\nВыведем распределение 2 признаков: Size и Ripeness")

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1) # 1 строка, 2 столбца, первый график
plt.hist(df['Size'], bins=100, color='skyblue')
plt.title('Распределение Size')
plt.subplot(1, 2, 2) # 1 строка, 2 столбца, второй график
plt.hist(df['Ripeness'], bins=100, color='orange')
plt.title('Распределение Ripeness')
plt.tight_layout()
plt.show()

# Тест Шапиро-Уилка - это статистический тест на нормальность распределения данных.
# Его основная цель - проверить гипотезу о том, что набор данных был взят из нормального распределения.

# Тест Шапиро-Вилка для столбца Size
stat_size, p_value_size = shapiro(df['Size'])
print(f"Тест Шапиро-Вилка для 'Size': статистика = {stat_size}, p-value = {p_value_size}")

# Тест Шапиро-Вилка для столбца Ripeness
stat_ripeness, p_value_ripeness = shapiro(df['Ripeness'])
print(f"Тест Шапиро-Вилка для 'Ripeness': статистика = {stat_ripeness}, p-value = {p_value_ripeness}")

print(f"\nДанные нормально распределены, т.к. p-value > 0,05\n")

size = df['Size']
ripeness = df['Ripeness']

# Тест Левена используется для проверки гомогенности дисперсий между группами данных.
# Гомогенность дисперсий означает, что дисперсии (или изменчивость) значений в каждой группе данных примерно одинаковы.
# Тест Левена формулируется как нулевая гипотеза о том, что дисперсии во всех группах равны.
# Альтернативная гипотеза предполагает, что хотя бы в одной из групп дисперсия отличается от дисперсий в других группах.

# Тест Левена для тех же признаков
stat, p = levene(size, ripeness)
print(f"Тест Левена: {stat}, p-value: {p}\n")

print(f"Так как p-value < 0,05 мы можем сделать вывод, что существует дисперсия между фичами\n")

print(f"{Fore.RED}Т-тест Уэлча (дисперсия разная, независимые выборки){Style.RESET_ALL}\n")

# Тест Уэлча - это статистический тест, используемый
# для сравнения средних значений двух независимых выборок, которые могут иметь разные дисперсии.
# Этот тест является модификацией t-теста Стьюдента и более устойчив к нарушению предположения о равенстве дисперсий между группами.

# T-тест Уэлча
t_stat, p_value = ttest_ind(size, ripeness, equal_var=False)

print(f"T-статистика Уэлча: {t_stat}, p-value: {p_value}\n")

# Расчёт t-тест Уэлча "вручную"

x1 = df['Size']
x2 = df['Ripeness']

x1_mean, x2_mean = x1.mean(), x2.mean() # Вычисляются средние значения для каждой из выборок

s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1) # Вычисляются выборочные дисперсии для каждой выборки
# Параметр ddof=1 указывает на то, что используется делитель n-1 для вычисления выборочной дисперсии, а не n

n1, n2 = x1.size, x2.size # Определяются размеры (число элементов) каждой выборки

t_stat_ind = (x1_mean - x2_mean) / ((s1 / n1 + s2 / n2) ** 0.5) # Рассчитывается значение t-теста Уэлча по формуле,
# где числитель - разность средних выборок, а знаменатель - корень из суммы отношений выборочных дисперсий к размерам выборок

print(f'Т-статистика Уэлча: {t_stat_ind}\n')

print(f"Из такого значения t-статистики можно сделать вывод, что нулевая гипотеза о равенстве средних значений двух независимых выборок не соблюдается и ее нужно отклонить\n")

print(f"{Fore.RED}Непарный т-тест (дисперсия гомогенная){Style.RESET_ALL}\n")

## Попытка найти столбцы с гомогенной дисперсией для проведения сравнения
size = df['Size']
ripeness = df['Sweetness']

# Тест Левена
stat, p = levene(size, ripeness)

print(f"Тест Левена: {stat}, p-value: {p}")

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1) # 1 строка, 2 столбца, первый подграфик
plt.hist(df['Size'], bins=100, color='skyblue')
plt.title('Распределение Size')

plt.subplot(1, 2, 2) # 1 строка, 2 столбца, второй подграфик
plt.hist(df['Sweetness'], bins=100, color='orange')
plt.title('Распределение Sweetness')

plt.tight_layout()
plt.show()

# Тест Шапиро-Вилка для 'Size'
stat_size, p_value_size = shapiro(df['Size'])
print(f"Тест Шапиро-Вилка для 'Size': статистика = {stat_size}, p-value = {p_value_size}\n")

# Тест Шапиро-Вилка для 'Ripeness'
stat_ripeness, p_value_ripeness = shapiro(df['Sweetness'])
print(f"Тест Шапиро-Вилка для 'Sweetness': статистика = {stat_ripeness}, p-value = {p_value_ripeness}\n")

# Т-тест с equal_var=True (при таком значении переменной, даем понять функции, что дисперсия гомогенна)

t_stat, p_value = ttest_ind(size, ripeness, equal_var=True)

print(f"T-статистика: {t_stat}, p-value: {p_value}\n")

# Расчёт т-теста "вручную"

x1 = df['Size']
x2 = df['Sweetness']

x1_mean, x2_mean = x1.mean(), x2.mean()

s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)

n1, n2 = x1.size, x2.size

t_stat_ind = (x1_mean - x2_mean) / ((s1 / n1 + s2 / n2) ** 0.5)

print(f'Т-статистика: {t_stat_ind}\n')

print(f'Из такого значения t-статистики можно сделать вывод, что статистически значимых различий'
      f' между средними значениями обнаружить не удалось, нулевая гипотеза подтверждена\n')

print(f"{Fore.RED}Парный т-тест{Style.RESET_ALL}\n")

t_stat, p_value = ttest_rel(df['Size'], df['Ripeness'])

print(f"Парный t-test: {t_stat}, p-value: {p_value}")

# Расчёт t-теста "вручную"
# Формула для расчета: x_ср / (std / sqrt(n))

differences = df['Size'] - df['Ripeness'] # Расчёт разностей между парами
mean_differences = differences.mean() # Расчёт среднего разностей
std_dev_difference = differences.std(ddof=1) # Расчёт стандартного отклонения разностей
n = differences.size # Расчёт объема выборки
t_stat_manual = mean_differences / (std_dev_difference / np.sqrt(n)) # Расчёт Т-теста

print(f"Парный t-test (вручную): {t_stat_manual}\n")

print(f"На основании полученного результата теста, мы можем отклонить нулевую гипотезу о равенстве средних значений в обеих группах\n")

print(f"{Fore.RED}One-way ANOVA (однофакторный дисперсионный анализ){Style.RESET_ALL}\n")
# Это статистический метод, который используется для сравнения средних значений трех или более групп данных,
# чтобы определить, существует ли статистически значимая разница между ними.
# В однофакторном дисперсионном анализе существует один фактор, который определяет группы (например, тип лекарства),
# и одна зависимая переменная (например, эффективность лечения).
# Анализ проводится путем сравнения вариабельности между группами с общей вариабельностью в данных.

x1 = df['Size']
x2 = df['Ripeness']
x3 = df['Sweetness']

# F-значение (F-value) - это статистика, которая измеряет различия между средними значениями
# в группах данных в однофакторном дисперсионном анализе.
# Высокое значение F-статистики может указывать на то, что средние значения
# в какой-то из групп значимо отличаются от средних значений в других группах.
# Однако для того чтобы утверждать статистическую значимость различий, также необходимо учитывать p-value.
# Если p-value меньше заданного уровня значимости, то различия считаются статистически значимыми.
f_value, p_value = stats.f_oneway(x1, x2, x3)
print(f"F-значение: {f_value}, p-value: {p_value}\n")

# Расчёт "вручную"

xx1 = df['Size']
xx2 = df['Ripeness']
xx3 = df['Sweetness']
data = [xx1, xx2, xx3]
overall_mean = np.mean(np.concatenate(data)) # Общее среднее
sst = sum([(value - overall_mean)**2 for group in data for value in group]) # Сумма квадратов общего отклонения (SST)
ssb = sum([len(group) * (np.mean(group) - overall_mean)**2 for group in data]) # Cумма квадратов между группами (SSB)
ssw = sst - ssb # Cумма квадратов внутри групп (SSW)
# Cтепени свободы
df_between = len(data) - 1  # Для SSB
df_within = sum([len(group) - 1 for group in data])  # Для SSW
# Cредние квадраты
msb = ssb / df_between
msw = ssw / df_within
# F-статистика
f_statistic = msb / msw

print(f"F-значение (вручную): {(f_statistic)}\n")

print(f"На основании полученного p-value, мы можем утверждать о наличии значимых различий между средними значениями групп\n")

print(f"{Fore.RED}Two-way ANOVA (двухфакторный дисперсионный анализ){Style.RESET_ALL}\n")
# Это статистический метод, который используется для анализа влияния двух категориальных факторов
# (независимых переменных) на зависимую переменную. Это расширение однофакторного дисперсионного анализа (ANOVA) для ситуаций,
# когда есть два фактора, влияющих на результаты исследования.

# Подготовка данных
df_long = pd.concat([pd.DataFrame({'Value': df['Size'], 'Group': 'Size'}), pd.DataFrame({'Value': df['Ripeness'], 'Group': 'Ripeness'})])

# Построение модели и выполнение ANOVA
model = ols('Value ~ C(Group)', data=df_long).fit()
anova_results = sm.stats.anova_lm(model, typ=1)

print(anova_results)

print(f"На основании полученного p-value, мы можем утверждать о наличии значимых различий между средними значениями групп\n")

print(f"{Fore.RED}Хи-квадрат{Style.RESET_ALL}\n")
# Это статистический тест, который используется для определения статистической значимости между наблюдаемыми частотами
# и ожидаемыми частотами в одной или нескольких категориях данных.
# Суть теста заключается в сравнении фактических частот (наблюдаемых) с частотами,
# которые были бы ожидаемы в случае, если никакой связи между переменными не существует (нулевая гипотеза).

# Для расчета хи-квадрата необходимо наличие двух категориальных переменных. Мы уже имеем "Quality" и создадим новую из "Juiciness"

bins = [-np.inf, -3, 0, 3, np.inf] # Определяем границы категории
df['Juiciness_Numerical'] = pd.cut(df['Juiciness'], bins=bins, labels=[0, 1, 2, 3]) # Создаем новый столбец сразу с числовыми метками категорий
print(df[['Juiciness', 'Juiciness_Numerical']].head())

# Создали, теперь можно проводить тест
cross_tab = pd.crosstab(df['Juiciness_Numerical'], df['Quality']) # Cоздаем кросс-таблицу для переменных 'Juiciness_Numerical' и 'Quality'
Xi2_stat, p_value, df, exp_freq = chi2_contingency(cross_tab) # Критерий хи-квадрат

print(f"\nХи-квадрат: {Xi2_stat}, p-value: {p_value}, степени свободы: {df}\n")
print("Ожидаемые частоты:")
print(exp_freq)

print(f"На основании полученного p-value, мы можем отклонить нулевую гипотезу о том, что статистической значимости между категориями нет, значит, между категориям присуствует статистически значимая связь\n")

cross_tab

obs_freq = np.array([
    [133, 20],
    [864, 527],
    [854, 1225],
    [145, 232]
])

N = obs_freq.sum() # Общее количество наблюдений
row_sums = obs_freq.sum(axis=1) # Cуммы по строкам
col_sums = obs_freq.sum(axis=0) # Cуммы по столбцам
exp_freq = np.outer(row_sums, col_sums) / N # Ожидаемые частоты
Xi2_stat_manual = ((obs_freq - exp_freq) ** 2 / exp_freq).sum() # Хи-квадрат статистику

print(f"Хи-квадрат (вручную): {Xi2_stat_manual}, ожидаемые частоты:\n {exp_freq}\n")

print(f"{Fore.RED}Критерий Фишера{Style.RESET_ALL}\n")
# Критерий Фишера, также известный как точный тест Фишера, это статистический тест,
# который используется для оценки статистической значимости ассоциации между двумя категориальными переменными,
# особенно в случае малых выборок данных.
# Суть теста заключается в сравнении наблюдаемого распределения частот в кросс-таблице с тем,
# которое ожидается в случае, если переменные были бы независимы.
# Нулевая гипотеза предполагает отсутствие ассоциации между переменными,
# альтернативная гипотеза предполагает наличие такой ассоциации.

table_2x2 = np.array([
    [133, 20],
    [864, 527]
])
odds_ratio, p_value = fisher_exact(table_2x2)
print(f"Отношение шансов: {odds_ratio}, p-value: {p_value}\n")

# Для расчёта "вручную", формула вероятности следующая: ((a+b)!*(c+d)!*(a+c)!*(b+d)!)/(a!*b!*c!*d!*n!)
a = 133
b = 20
c = 864
d = 527
n = a + b + c + d

# Функция для расчёта вероятности
def calc_prob(a, b, c, d):
    return f(a + b) * f(c + d) * f(a + c) * f(b + d) / (f(a) * f(b) * f(c) * f(d) * f(a + b + c + d))
p_real = calc_prob(a, b, c, d)
p_values = []
for x in range(0, min(a + b, a + c) + 1):
    # Создание новых возможных переменных
    new_a = x
    new_b = (a + b) - x
    new_c = (a + c) - x
    new_d = (b + d) - (a + b) + x
    # Расчёт вероятности для каждой возможной таблицы
    p = calc_prob(new_a, new_b, new_c, new_d)
    # Сохраняем вероятности, если они <= наблюдаемой вероятности
    if p <= p_real:
        p_values.append(p)
p_value_manual = sum(p_values)
odds_ratio_manual = (133 * 527) / (20 * 864)
print(f"Отношение шансов (вручную): {odds_ratio_manual}, наблюдаемая вероятность: {p_real}, p-value: {p_value_manual}, отношение шансов: {odds_ratio_manual}")

ModuleNotFoundError: No module named 'colorama'